In [1]:
import pandas as pd

In [2]:
pa_latin_art = pd.read_csv('../data_samples/pa_latin_art.csv')
stored_latin_art = pa_latin_art.where(pa_latin_art.file_downloaded == True).dropna(how='all')
stored_latin_art.reset_index(drop = True, inplace=True)

In [3]:
stored_latin_art

,expanded_url,title,forwarddisplayname,medium,displaydate,sequence,width,height,maxpixels,depictstmsobjectid,...,provenancetext,creditline,classification,parentid,isvirtual,portfolio,series,file_name,directory,file_downloaded
0,https://api.nga.gov/iiif/95bc5221-79a1-49e1-b7...,Festival Andino,Nemesio Antunez,color lithograph,"Chilean, 1918 - 1993",0.0,2963.0,4000.0,640.0,46741.0,...,NaN,Rosenwald Collection,Print,NaN,0.0,NaN,NaN,Festival_Andino.jpg,./All_Images,True
1,https://api.nga.gov/iiif/f8a0c85d-7cfa-4e63-ae...,Chester Dale,Diego Rivera,oil on canvas,"Mexican, 1886 - 1957",0.0,5297.0,4171.0,640.0,46537.0,...,"Commissioned 1945 by Chester Dale [1883-1962],...",Chester Dale Collection,Painting,NaN,0.0,NaN,NaN,Chester_Dale.jpg,./All_Images,True
2,https://api.nga.gov/iiif/a2bc5dff-08a4-46d5-97...,Compostion XVI,Matta,"color etching, soft-ground, and aquatint with ...","Chilean, 1911 - 2002",0.0,2926.0,4000.0,640.0,48425.0,...,NaN,Rosenwald Collection,Print,NaN,0.0,"Suite from ""Come detta dentro vo significando""","Suite from ""Come detta dentro vo significando""",Compostion_XVI.jpg,./All_Images,True
3,https://api.nga.gov/iiif/c73c0136-d48d-4bea-89...,Compostion IV,Matta,"color etching, soft-ground, and aquatint","Chilean, 1911 - 2002",0.0,4000.0,3101.0,640.0,48413.0,...,NaN,Rosenwald Collection,Print,NaN,0.0,"Suite from ""Come detta dentro vo significando""","Suite from ""Come detta dentro vo significando""",Compostion_IV.jpg,./All_Images,True
4,https://api.nga.gov/iiif/1f1918c9-9d30-4491-a9...,Compostion I,Matta,"color etching, soft-ground, and aquatint on Ja...","Chilean, 1911 - 2002",0.0,4000.0,3084.0,640.0,48410.0,...,NaN,Rosenwald Collection,Print,NaN,0.0,"Suite from ""Come detta dentro vo significando""","Suite from ""Come detta dentro vo significando""",Compostion_I.jpg,./All_Images,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,https://api.nga.gov/iiif/552e99b7-0bb5-45e1-8b...,Gran Alarma Escandalosa,José Guadalupe Posada,metalcut and letterpress on pink newsprint,"Mexican, 1851 - 1913",0.0,4949.0,7513.0,NaN,152826.0,...,NaN,Given by Art Information Volunteers in Honor o...,Print,NaN,0.0,NaN,NaN,Gran_Alarma_Escandalosa_Posada.jpg,./All_Images,True
324,https://api.nga.gov/iiif/83c83210-819e-4c27-8c...,Serafina,José Guadalupe Posada,metalcut and letterpress on yellow newsprint,"Mexican, 1851 - 1913",0.0,2873.0,4000.0,NaN,152827.0,...,NaN,Given by Art Information Volunteers in Honor o...,Print,NaN,0.0,NaN,NaN,Serafina_Posada.jpg,./All_Images,True
325,https://api.nga.gov/iiif/92410def-988a-4866-9c...,La Mujer de Cien Maridos - Como Alfileres Pren...,José Guadalupe Posada,metalcut on red newsprint,"Mexican, 1851 - 1913",0.0,2726.0,4000.0,NaN,152829.0,...,NaN,Given by Art Information Volunteers in Honor o...,Print,NaN,0.0,NaN,NaN,La_Mujer_de_Cien_Maridos_-_Como_Alfileres_Pren...,./All_Images,True
326,https://api.nga.gov/iiif/fcbe84cb-faa8-471d-b4...,Don Chepito,José Guadalupe Posada,relief etching on pink paper,"Mexican, 1851 - 1913",0.0,4000.0,3169.0,NaN,163508.0,...,"Beverly and Christopher With, Washington, D.C....",Gift of Christopher and Beverly With in memory...,Print,NaN,0.0,NaN,NaN,Don_Chepito_Posada.jpg,./All_Images,True


### Feature Engineering

In [4]:
### Looking for text-heavy features in the data that give insight into the context of the image

In [5]:
text_dict = {}
for entry in stored_latin_art.title:
    entry_words = entry.split(' ')
    for word in entry_words:
        word = word.strip(')([],\./')
        word = word.lower()
        if word in text_dict.keys():
            text_dict[word] += 1
        else:
            text_dict[word] = 1

In [6]:
title_occurrences = pd.Series(text_dict).sort_values(ascending=False)

In [7]:
title_occurrences

untitled     46
the          46
of           33
garden       28
la           20
             ..
graceful      1
gay           1
somewhat      1
graciosa      1
maderista     1
Length: 536, dtype: int64

In [8]:
import torch
import torch.nn.functional as fun

In [9]:
titles = stored_latin_art.title.copy()

In [10]:
import re
def clean_words(text):
    def strip(string):
        return re.sub('(?:^[{\W}]+)|(?:[{\W}]+$)', '', string)
    text_arr = text.split(' ')
    words = []
    for word in text_arr:
        edited = strip(word)
        words.append(edited.lower())
    return ' '.join(words)

In [11]:
#cleaning and formatting the data to give uniformity and clarity
titles = titles.apply(clean_words)

In [12]:
from typing import Union, Iterable
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [13]:
train_text = iter(zip(titles.index, titles))

In [16]:
titles.to_csv('titles.csv', index = False)

In [14]:
voc = build_vocab_from_iterator(yield_tokens(train_text))

328lines [00:00, 88676.79lines/s]
